<a href="https://colab.research.google.com/github/yeonsub/models_from_scratch/blob/main/MCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

# PyMC 라이브러리 설치 (처음 실행 시 필요)
# 이미 설치되어 있다면 건너뛸 수 있습니다.
#!pip install pymc

# Parameters for two modes (전역 변수로 명시적으로 정의)
MU1 = 7.0
SIGMA1 = 2.0
MU2 = 15.0
SIGMA2 = 5.0

# Define two normal distributions for the target PDF
target_dist1 = ss.norm(loc=MU1, scale=SIGMA1)
target_dist2 = ss.norm(loc=MU2, scale=SIGMA2)

# Define the bimodal target distribution's PDF
class BimodalTarget:
    def pdf(self, x):
        return 0.5 * target_dist1.pdf(x) + 0.5 * target_dist2.pdf(x)

target = BimodalTarget() # target distribution

# Define a wider range for xs to cover both modes of the bimodal distribution
# This ensures xs and ys are defined when needed for plotting.
x_min_range = min(MU1 - 4*SIGMA1, MU2 - 4*SIGMA2) # Extend range slightly
x_max_range = max(MU1 + 4*SIGMA1, MU2 + 4*SIGMA2) # Extend range slightly
xs = np.linspace(x_min_range, x_max_range, 500) # Increased points for smoother PDF
ys = target.pdf(xs)

PyMC를 사용하여 MCMC 샘플링을 수행합니다. 이봉형 분포를 PyMC 모델로 정의하고 샘플링합니다.

In [ ]:
import pymc as pm

# PyMC 모델 정의
with pm.Model() as bimodal_model:
    # x는 샘플링할 확률 변수입니다. 초기 값은 MCMC의 시작점과 유사합니다.
    # pm.Flat을 사용하여 비정보적인 사전 분포를 부여하여 likelihood가 분포를 주도하도록 합니다.
    x_pymc = pm.Flat('x_pymc', initval=3.0)

    # 이봉형 분포의 로그 확률 밀도를 PyMC의 함수를 사용하여 직접 정의합니다.
    # 전역 변수 MU1, SIGMA1, MU2, SIGMA2를 사용합니다.
    log_prob_dist1 = pm.logp(pm.Normal.dist(mu=MU1, sigma=SIGMA1), x_pymc)
    log_prob_dist2 = pm.logp(pm.Normal.dist(mu=MU2, sigma=SIGMA2), x_pymc)

    # 두 정규 분포의 확률을 가중 평균하여 이봉형 분포의 확률을 계산하고 로그를 취합니다.
    # log(0.5 * p1 + 0.5 * p2) = log(0.5 * exp(log_p1) + 0.5 * exp(log_p2))
    pm.Potential('target_potential', pm.math.log(0.5 * pm.math.exp(log_prob_dist1) + 0.5 * pm.math.exp(log_prob_dist2)))

    # MCMC 샘플링 실행
    # draws: 샘플링할 개수
    # tune: 버닝 기간 (수렴을 위한 초기 단계)
    # chains: 병렬로 실행할 MCMC 체인 수
    trace = pm.sample(draws=10000, tune=2000, chains=2, random_seed=42, return_inferencedata=True)

# 샘플링 결과 추출
x_sampled_pymc = trace.posterior['x_pymc'].values.flatten()

# PyMC로 샘플링한 결과 시각화
plt.figure(figsize=(10, 6))
plt.hist(x_sampled_pymc, bins='fd', density=True, alpha=0.6, color='green', label='PyMC Sampled (MCMC)')
# Use the newly defined xs and ys for the target PDF plot
plt.plot(xs, ys, color='navy', label='Target PDF')
plt.title('MCMC Sampling with PyMC for Bimodal Distribution')
plt.xlabel('X')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()